In [1]:
# 📌 1. Import Library
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.pagesizes import A4
import plotly.io as pio

# Styling
pio.templates.default = "plotly_white"
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

df = pd.read_csv('../data/owid-energy-data.csv')

In [2]:
# --- A. DEFINISI PARAMETER & MUAT DATA ---
# Daftar negara target
g7_countries = ['United States', 'Canada', 'Germany', 'United Kingdom', 'France', 'Italy', 'Japan']
brics_countries = ['Brazil', 'Russia', 'India', 'China', 'South Africa']
target_countries = g7_countries + brics_countries
start_year = 2000
end_year = 2022

In [3]:
# Kolom kunci untuk analisis (minimalisasi missing values)
key_columns = [
    'country', 'year', 'gdp', 'population', 
    'fossil_share_energy', 'fossil_fuel_consumption', # Tambahan: Konsumsi Absolut
    'low_carbon_share_energy', 
    'solar_consumption', 'wind_consumption', 
    'energy_per_gdp'
]

In [4]:
df.head()

,country,year,iso_code,population,gdp,biofuel_cons_change_pct,biofuel_cons_change_twh,biofuel_cons_per_capita,biofuel_consumption,biofuel_elec_per_capita,...,solar_share_elec,solar_share_energy,wind_cons_change_pct,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_electricity,wind_energy_per_capita,wind_share_elec,wind_share_energy
0,ASEAN (Ember),2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
1,ASEAN (Ember),2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
2,ASEAN (Ember),2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
3,ASEAN (Ember),2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
4,ASEAN (Ember),2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN


In [5]:
df.info()
df.isnull().sum().sort_values(ascending=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23195 entries, 0 to 23194
Columns: 130 entries, country to wind_share_energy
dtypes: float64(127), int64(1), object(2)
memory usage: 23.0+ MB


biofuel_cons_change_pct             21125
solar_cons_change_pct               20680
nuclear_cons_change_pct             20616
wind_cons_change_pct                20397
other_renewables_cons_change_pct    19207
                                    ...  
oil_prod_change_twh                  5466
oil_production                       5203
population                           4466
year                                    0
country                                 0
Length: 130, dtype: int64

In [6]:
print("--- Data Cleaning & Preparation ---")

# 1. Filter Negara Target dan Tahun
df_filtered = df[
    (df['country'].isin(target_countries)) & 
    (df['year'] >= start_year) & 
    (df['year'] <= end_year)
].copy()

--- Data Cleaning & Preparation ---


In [7]:
# 2. Seleksi Kolom Kunci
df_clean = df_filtered[key_columns].copy()

In [8]:
# 3. Penanganan Nilai Hilang (Interpolasi GDP)
# Mengisi gap kecil GDP menggunakan interpolasi linier (within each country)
df_clean['gdp'] = df_clean.groupby('country')['gdp'].transform(lambda x: x.interpolate(method='linear'))

In [9]:
# 4. Membuang sisa baris yang masih memiliki NaN pada kolom kunci utama
df_clean.dropna(subset=['gdp', 'fossil_share_energy', 'low_carbon_share_energy', 'energy_per_gdp'], inplace=True)

In [10]:
# 5. Membuat Kolom Kelompok Negara
def assign_group(country):
    return 'G7' if country in g7_countries else 'BRICS'

df_clean['Group'] = df_clean['country'].apply(assign_group)

print(f"Data bersih siap: {len(df_clean)} baris.")
print("-" * 35)

Data bersih siap: 276 baris.
-----------------------------------


In [11]:
# --- C. LANGKAH 3: ANALISIS DAN VISUALISASI NARASI ---

# NARASI 1: LAJU DEKARBONISASI (THE SHIFT)
# Bandingkan tren rata-rata pangsa energi fosil

print("\n--- Narasi 1: Laju Dekarbonisasi (Fosil Share) ---")

df_group_trend = df_clean.groupby(['Group', 'year'])['fossil_share_energy'].mean().reset_index()

fig1 = px.line(
    df_group_trend,
    x='year',
    y='fossil_share_energy',
    color='Group',
    title='Tren Rata-Rata Pangsa Energi Fosil (2000-2022)',
    labels={'fossil_share_energy': 'Pangsa Fosil (%)', 'year': 'Tahun'},
    markers=True
)
fig1.update_layout(yaxis_title="Pangsa Energi Fosil (%)", xaxis_title="Tahun")
fig1.show()


--- Narasi 1: Laju Dekarbonisasi (Fosil Share) ---


In [12]:
# NARASI 2: PERTUMBUHAN ENERGI TERBARUKAN (THE GROWTH ENGINE)
print("\n--- Narasi 2: Pertumbuhan Energi Terbarukan Baru ---")

df_growth = df_clean[(df_clean['year'] == 2012) | (df_clean['year'] == 2022)].copy()
df_pivot = df_growth.pivot_table(
    index=['country', 'Group'],
    columns='year',
    values=['solar_consumption', 'wind_consumption']
).reset_index()

# PERBAIKAN: Flattening MultiIndex columns
# Mengubah tuple kolom seperti ('solar_consumption', 2022) menjadi string biasa
df_pivot.columns = ['_'.join(map(str, col)).strip('_') if col[1] != '' else col[0] for col in df_pivot.columns.values]

# Menghitung Pertumbuhan Absolut (TWh) menggunakan nama kolom yang sudah diperbaiki
df_pivot['Solar_Growth_TWh'] = df_pivot['solar_consumption_2022'] - df_pivot['solar_consumption_2012']
df_pivot['Wind_Growth_TWh'] = df_pivot['wind_consumption_2022'] - df_pivot['wind_consumption_2012']

# Visualisasi Pertumbuhan Solar
fig2 = px.bar(
    df_pivot.sort_values('Solar_Growth_TWh', ascending=False),
    x='country',
    y='Solar_Growth_TWh',
    color='Group',
    title='Pertumbuhan Konsumsi Energi Surya Absolut (2012-2022)',
    labels={'Solar_Growth_TWh': 'Pertumbuhan Konsumsi Surya (TWh)', 'country': 'Country'},
    text_auto='.2s'
)
fig2.update_layout(yaxis_title="Pertumbuhan Konsumsi Surya (TWh)", xaxis_title="Negara")
fig2.show() # Tampilkan visualisasi


--- Narasi 2: Pertumbuhan Energi Terbarukan Baru ---


In [13]:
# NARASI 3: EFISIENSI ENERGI (THE SUSTAINABILITY TEST)
# Bandingkan tren rata-rata efisiensi (energy_per_gdp)

print("\n--- Narasi 3: Tren Efisiensi Energi ---")

df_efficiency_trend = df_clean.groupby(['Group', 'year'])['energy_per_gdp'].mean().reset_index()

fig3 = px.line(
    df_efficiency_trend,
    x='year',
    y='energy_per_gdp',
    color='Group',
    title='Tren Rata-Rata Intensitas Energi (Energy per GDP)',
    labels={'energy_per_gdp': 'Energy per GDP (koe/$)', 'year': 'Tahun'},
    markers=True
)
fig3.update_layout(
    yaxis_title="Energy per GDP (koe/$) - Semakin Rendah, Semakin Efisien", 
    xaxis_title="Tahun"
)
fig3.show()


--- Narasi 3: Tren Efisiensi Energi ---


In [14]:
# --- B. PEMBERSIHAN (Langkah Ulang + Tambahan Kolom) ---
df_filtered = df[
    (df['country'].isin(target_countries)) & 
    (df['year'] >= start_year) & 
    (df['year'] <= end_year)
].copy()

df_clean_ext = df_filtered[key_columns].copy()
df_clean_ext['gdp'] = df_clean_ext.groupby('country')['gdp'].transform(lambda x: x.interpolate(method='linear'))

# Menambahkan kolom Group
def assign_group(country):
    return 'G7' if country in g7_countries else 'BRICS'

df_clean_ext['Group'] = df_clean_ext['country'].apply(assign_group)

# Cleaning: Hapus baris yang tidak memiliki data konsumsi absolut
df_absolute = df_clean_ext.dropna(subset=['fossil_fuel_consumption']).copy()

In [15]:
# --- C. ANALISIS KONSUMSI FOSIL ABSOLUT ---

print("--- Analisis Konsumsi Energi Fosil Absolut (TWh) ---")

# Hitung Total Konsumsi Fosil Rata-Rata per Kelompok
df_fossil_absolute_trend = df_absolute.groupby(['Group', 'year'])['fossil_fuel_consumption'].sum().reset_index()

# Visualisasi Tren Konsumsi Fosil Absolut
fig_absolute_fossil = px.line(
    df_fossil_absolute_trend,
    x='year',
    y='fossil_fuel_consumption',
    color='Group',
    title='Total Konsumsi Energi Fosil (Absolut) G7 vs BRICS (2000-2022)',
    labels={'fossil_fuel_consumption': 'Konsumsi Fosil Total (TWh)', 'year': 'Tahun'},
    markers=True,
    line_shape='spline'
)
fig_absolute_fossil.update_layout(
    yaxis_title="Konsumsi Fosil Total (TWh)", 
    xaxis_title="Tahun"
)
fig_absolute_fossil.show()

# Perbandingan Konsumsi Total pada Tahun Terakhir
latest_year_data = df_fossil_absolute_trend[df_fossil_absolute_trend['year'] == end_year]

print(f"\nTotal Konsumsi Fosil (TWh) pada {end_year}:")
print(latest_year_data.pivot_table(index='year', columns='Group', values='fossil_fuel_consumption').round(0))

--- Analisis Konsumsi Energi Fosil Absolut (TWh) ---



Total Konsumsi Fosil (TWh) pada 2022:
Group    BRICS       G7
year                   
2022   55991.0  35148.0
